In [339]:
### IMPORT PACKAGES ###

import pandas as pd
import numpy as np

In [340]:
### CREATING DATA SOURCE ###

# Import data 
data_apr_27_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190427.txt")
data_may_04_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190504.txt")
data_may_11_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190511.txt")
data_may_18_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190518.txt")
data_may_25_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190525.txt")
data_jun_06_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190601.txt")
data_jun_08_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190608.txt")
data_jun_15_19 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190615.txt")

# Combine data
all_data = pd.concat([data_apr_27_19, data_may_04_19, data_may_11_19, data_may_18_19, data_may_25_19
         , data_jun_06_19, data_jun_08_19, data_jun_15_19 ], ignore_index=True)

# Check that data imported successfully
all_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1640586 entries, 0 to 1640585
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count    Dtype 
---  ------                                                                --------------    ----- 
 0   C/A                                                                   1640586 non-null  object
 1   UNIT                                                                  1640586 non-null  object
 2   SCP                                                                   1640586 non-null  object
 3   STATION                                                               1640586 non-null  object
 4   LINENAME                                                              1640586 non-null  object
 5   DIVISION                                                              1640586 non-null  object
 6   DATE                                                                  1640586 non-

In [342]:
### FILTERING, CLEANING, AND PREPARING DATA ###

# Filter data to only Regular scheduled audit events to remove any possible irregularities from recovered audits
all_data = all_data[all_data['DESC'] == 'REGULAR']

# Clean column names
all_data.columns = [column.strip() for column in all_data.columns]

# Create new variables to make datetime and turnstile easier to work with
all_data['DATETIME'] = pd.to_datetime(all_data['DATE'] + ' ' + all_data['TIME'])
all_data['DAYOFWEEK'] = all_data['DATETIME'].dt.day_name()
all_data['TURNSTILE'] = all_data['C/A'] + all_data['UNIT'] + all_data['SCP'] + all_data['STATION']

# Sort data so it is in order of turnstile and datetime
all_data.sort_values(by = ['TURNSTILE', 'DATETIME'], inplace = True)

# Check for duplicate turnstile/datetime combinations
(all_data
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(10)

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2019-04-20 00:00:00,1
1089203,R142,R293,01-06-00,34 ST-PENN STA,2019-05-12 02:00:00,1
1089201,R142,R293,01-06-00,34 ST-PENN STA,2019-05-11 18:00:00,1
1089200,R142,R293,01-06-00,34 ST-PENN STA,2019-05-11 14:00:00,1
1089199,R142,R293,01-06-00,34 ST-PENN STA,2019-05-11 10:00:00,1
1089198,R142,R293,01-06-00,34 ST-PENN STA,2019-05-11 06:00:00,1
1089197,R142,R293,01-06-00,34 ST-PENN STA,2019-05-11 02:00:00,1
1089196,R142,R293,01-06-00,34 ST-PENN STA,2019-05-10 22:00:00,1
1089195,R142,R293,01-06-00,34 ST-PENN STA,2019-05-10 18:00:00,1
1089194,R142,R293,01-06-00,34 ST-PENN STA,2019-05-10 14:00:00,1


In [345]:
### CHECKING FOR IRREGULARITIES IN CUMULATIVE ENTRIES DATA ###

# Aggregate data to daily level
turnstiles_daily = (all_data_clean
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.first())

# Calculate previous data and previous entries
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily.groupby(["C/A", "UNIT", "SCP", "STATION"])
                        ["DATE", "ENTRIES"].apply(lambda grp: grp.shift(1)))

# Check to see if entries from previous day are ever 
(turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]
                        .groupby(["C/A", "UNIT", "SCP", "STATION"])
                        .size())


<ipython-input-345-7252ad492d9a>:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily.groupby(["C/A", "UNIT", "SCP", "STATION"])


C/A   UNIT  SCP       STATION        
A025  R023  01-03-01  34 ST-HERALD SQ    55
A038  R085  00-00-01  8 ST-NYU            1
A043  R462  00-06-00  CANAL ST            1
A046  R463  00-00-03  CANAL ST            1
A049  R088  02-05-00  CORTLANDT ST       42
                                         ..
R621  R060  00-03-01  EASTN PKWY-MUSM     2
R622  R123  00-00-00  FRANKLIN AV        55
R629  R065  00-00-00  ROCKAWAY AV         1
R646  R110  01-00-01  FLATBUSH AV-B.C    55
R730  R431  00-00-04  EASTCHSTER/DYRE    51
Length: 481, dtype: int64

In [351]:
### TRANSFORMING CUMULATIVE ENTRIES AND EXITS TO NET ENTRIES AND EXITS ###
    ## Due to instances of previous daily turnstile entries that were higher than the current daily entries 
    ## (see cell above), there was evidence that some counters were going backwards. 
    ## Therefore, we used absolute value in net calculations

# Resort data to ensure it is in correct order
all_data.sort_values(by = ['TURNSTILE', 'DATETIME'], inplace = True)

# Net entries and net exit calculations using absolute value
all_data['NET_ENTRIES'] = all_data.groupby('TURNSTILE')['ENTRIES'].diff().abs()
all_data['NET_EXITS'] = all_data.groupby('TURNSTILE')['EXITS'].diff().abs()

# Removing rows with entries or exits above 10,000 since anything over 10,000 entries or exits means that 
# at least 2,500 people went through that turnstile every hour for four hours straight and that seems unrealistic
indexNames = all_data[(all_data['NET_ENTRIES'] > 10000 ) | (all_data['NET_EXITS'] > 10000)].index
all_data_clean = all_data.drop(indexNames)

# Combine net entries and net exits into variable for all foot traffic
all_data_clean['TOTAL_FOOT_TRAFFIC'] = all_data_clean['NET_ENTRIES'] + all_data_clean['NET_EXITS']

# Verify data looks correct
all_data_clean.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAYOFWEEK,TURNSTILE,NET_ENTRIES,NET_EXITS,TOTAL_FOOT_TRAFFIC
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/20/2019,00:00:00,REGULAR,7026702,2382234,2019-04-20 00:00:00,Saturday,A002R05102-00-0059 ST,NaN,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/20/2019,04:00:00,REGULAR,7026719,2382240,2019-04-20 04:00:00,Saturday,A002R05102-00-0059 ST,17.0,6.0,23.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/20/2019,08:00:00,REGULAR,7026737,2382269,2019-04-20 08:00:00,Saturday,A002R05102-00-0059 ST,18.0,29.0,47.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/20/2019,12:00:00,REGULAR,7026814,2382335,2019-04-20 12:00:00,Saturday,A002R05102-00-0059 ST,77.0,66.0,143.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/20/2019,16:00:00,REGULAR,7027007,2382395,2019-04-20 16:00:00,Saturday,A002R05102-00-0059 ST,193.0,60.0,253.0


In [352]:
# Code used for tables 
all_data_clean.groupby(['STATION', 'DATE']).sum(['NET_ENTRIES', 'NET_EXITS']).sort_values('NET_ENTRIES', ascending = False)

df_weekday = all_data_clean.groupby(['STATION']).sum(['TOTAL_FOOT_TRAFFIC'])

df_weekday.sort_values(['TOTAL_FOOT_TRAFFIC'], ascending = False).head(50)

,ENTRIES,EXITS,NET_ENTRIES,NET_EXITS,TOTAL_FOOT_TRAFFIC
STATION,,,,,
34 ST-PENN STA,1320184649158,1395096774001,8215058.0,7551130.0,15766188.0
GRD CNTRL-42 ST,883735077669,1101475879710,7162035.0,6568945.0,13730980.0
34 ST-HERALD SQ,1566137770660,1963907924717,6129269.0,5610908.0,11740177.0
TIMES SQ-42 ST,2076660452334,1599271654399,5103711.0,4735188.0,9838899.0
14 ST-UNION SQ,240992656454,126495142802,5091185.0,4716003.0,9807188.0
23 ST,1688831497641,1457910805183,5490025.0,3956051.0,9446076.0
42 ST-PORT AUTH,2641286638869,2161924758535,5243505.0,4172865.0,9416370.0
FULTON ST,1541282674973,1574881353108,4625623.0,3920938.0,8546561.0
86 ST,827928707815,893301512906,4256258.0,3897890.0,8154148.0
